In [1]:
!pip install torch transformers


In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW


 Example text data (you can replace this with any larger corpus) text = """ Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her. [CO5]

 (i) Build the Transformer Model on above dataset

In [11]:
# Sample text data
text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother,
who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother.
On her way through the woods, she met a big bad wolf who wanted to eat her."""

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Tokenize the text
tokens = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
input_ids = tokens.input_ids


In [4]:
# Initialize model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.train()  # Set the model to training mode


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

(ii) Train the model using 20, 60, 70 epochs



In [5]:
# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define training function
def train_model(model, input_ids, epochs):
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss.item()}")

# Train the model for 20 epochs, then 60, then 70
train_model(model, input_ids, epochs=20)
train_model(model, input_ids, epochs=60)
train_model(model, input_ids, epochs=70)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20 - Loss: 3.2264277935028076
Epoch 2/20 - Loss: 2.716520309448242
Epoch 3/20 - Loss: 2.221055269241333
Epoch 4/20 - Loss: 1.8699105978012085
Epoch 5/20 - Loss: 1.5087488889694214
Epoch 6/20 - Loss: 1.173133134841919
Epoch 7/20 - Loss: 1.0864776372909546
Epoch 8/20 - Loss: 0.8180212378501892
Epoch 9/20 - Loss: 0.7951277494430542
Epoch 10/20 - Loss: 0.5697635412216187
Epoch 11/20 - Loss: 0.5774595737457275
Epoch 12/20 - Loss: 0.45084086060523987
Epoch 13/20 - Loss: 0.2772672176361084
Epoch 14/20 - Loss: 0.324715256690979
Epoch 15/20 - Loss: 0.12335899472236633
Epoch 16/20 - Loss: 0.116624616086483
Epoch 17/20 - Loss: 0.06598766148090363
Epoch 18/20 - Loss: 0.18738755583763123
Epoch 19/20 - Loss: 0.1616302728652954
Epoch 20/20 - Loss: 0.15020056068897247
Epoch 1/60 - Loss: 0.02349456399679184
Epoch 2/60 - Loss: 0.01170744001865387
Epoch 3/60 - Loss: 0.06775382906198502
Epoch 4/60 - Loss: 0.019236678257584572
Epoch 5/60 - Loss: 0.01694214902818203
Epoch 6/60 - Loss: 0.008048617281

 (iii) After training, use the model to generate new text by feeding it an initial seed text

In [8]:
def generate_text(model, tokenizer, seed_text, max_length=70):
    model.eval()
    input_ids = tokenizer.encode(seed_text, return_tensors="pt")
    with torch.no_grad():
        generated_ids = model.generate(input_ids, max_length=max_length)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

seed_text = "Once upon a time"
generated_text = generate_text(model, tokenizer, seed_text)
print("Generated Text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, 
who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. 
On her way through the woods, she met a big bad wolf who wanted to eat her. 

